In [6]:

with open("SimpleStorage.sol","r") as file:
    simple_storage_file = file.read()
    #print(simple_storage_file)

# Steps
pip install web3
pip install black
pip install py-solc-x
solcx.install_solc('0.6.0')

In [25]:
from solcx import compile_standard
import json
from web3 import Web3

compiled_sol = compile_standard(
    {
    "language": "Solidity",
    "sources": {"SimpleStorage.sol" : {"content": simple_storage_file}},
    "settings": {
        "outputSelection" : {
            "*" : {
                "*" : ["abi","metadata","evm.bytecode","evm.sourceMap"]}
        }
    },
    },
    solc_version = "0.6.0"
    )

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

#get bytecode
bytecode = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["evm"]["bytecode"]["object"]
#get abi
abi = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["abi"]

#connecting to ganache
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
my_address = "0xa12918D5a0E18b46a55f53667c0E5546FF0B5F60"
private_key = "0x7934ba9598bf0ceb895582fc759078226375f8f3aa974e7fbb83303e11509503"

#Create Contract in Python
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)
print(SimpleStorage)

#Get the latest transaction
nonce = w3.eth.getTransactionCount(my_address)
#Build Transaction
transaction = SimpleStorage.constructor().buildTransaction({"chainId":chain_id,"gasPrice": w3.eth.gas_price,"from":my_address, "nonce": nonce})
#Sign it
signed_txn = w3.eth.account.sign_transaction(transaction,private_key=private_key)
#Send it
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction) 

<class 'web3._utils.datatypes.Contract'>


In [23]:
signed_txn

SignedTransaction(rawTransaction=HexBytes('0xf9061f808504a817c800830596278080b905ca608060405234801561001057600080fd5b506105aa806100206000396000f3fe608060405234801561001057600080fd5b50600436106100575760003560e01c80632e64cec11461005c5780636057361d1461007a5780636f760f41146100a85780638bab8dd51461016d5780639e7a13ad1461023c575b600080fd5b6100646102ea565b6040518082815260200191505060405180910390f35b6100a66004803603602081101561009057600080fd5b81019080803590602001909291905050506102f3565b005b61016b600480360360408110156100be57600080fd5b81019080803590602001906401000000008111156100db57600080fd5b8201836020820111156100ed57600080fd5b8035906020019184600183028401116401000000008311171561010f57600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290803590602001909291905050506102fd565b005b6102266004803603602081101561018357600080fd5b81019080803590602001906401000000008111156101a057600080fd5b8201836020820111156